# price-api

Get prices from api, and calculate outperformance compared to indexes. 

In [1]:
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import requests, json

## Using fix_yahoo_finance

In [2]:
from pandas_datareader import data as pdr

import fix_yahoo_finance as yf
yf.pdr_override() # <== that's all it takes :-)

### Testing

In [ ]:
# Test on one ticker
data_fix1 = pdr.get_data_yahoo("BOO.L", start="2016-06-06", end="2017-09-05")

data_fix1.shape
data_fix1.sort_index(ascending=False).head()

In [ ]:
# Limit to just the adjusted close, for start and end dates

adj_close = data_fix1[['Adj Close']]

adj_close.head()
adj_close.tail()

summary_df = pd.DataFrame()

summary_df['start'] = adj_close.iloc[0]

summary_df['end'] = adj_close.iloc[-1]

summary_df.head()

### Read in tickers and dates sheet

In [98]:
tickers = pd.read_csv('~/Downloads/report-dates - Master.csv', parse_dates=['AR Date', 'FY end', 'Period Start Date', 'Period End Date'])

tickers['Symbol Yahoo'] = tickers['Symbol'].astype(str) + ".L"

tickers.shape
tickers
tickers.dtypes

(54, 9)

,Company,Symbol,AR Date,FY end,YE to Report lag,Period Start Date,Period End Date,AR link,Symbol Yahoo
0,ABCAM,ABC,2016-11-09,2016-06-30,73,2016-09-25,2017-08-28,https://uploads.abcamplc.com/wp-content/upload...,ABC.L
1,ADVANCED MEDICAL SOLUTIONS,AMS,2017-03-14,2016-12-31,73,2017-03-28,2018-02-28,https://www.admedsol.com/wp-content/uploads/20...,AMS.L
2,ALLIANCE PHARMA,APH,2017-03-29,2016-12-31,88,2017-12-04,2018-03-15,http://investors.alliancepharmaceuticals.com/~...,APH.L
3,ASOS,ASC,2016-10-28,2016-08-31,58,2016-11-11,2017-10-14,https://www.asosplc.com/~/media/Files/A/Asos-V...,ASC.L
4,BLUE PRISM,PRSM,2017-01-24,2016-10-31,85,2017-07-02,2018-10-01,https://www.blueprism.com/wp-content/uploads/2...,PRSM.L
5,boohoo.com,BOO,2016-05-23,2016-02-29,84,2016-06-06,2017-09-05,http://www.boohooplc.com/~/media/Files/B/Booho...,BOO.L
6,BROOKS MACDONALD GROUP,BRK,2016-09-21,2016-06-30,83,2016-05-10,2017-07-09,https://www.brooksmacdonald.com/~/media/Files/...,BRK.L
7,BURFORD CAPITAL,BUR,2017-03-13,2016-12-31,72,2017-03-27,2018-02-27,http://www.burfordcapital.com/wp-content/uploa...,BUR.L
8,CAMELLIA,CAM,2017-02-04,2016-12-31,92,2017-04-16,2018-03-19,https://www.camellia.plc.uk/pages/8a7c0775-d4e...,CAM.L
9,CARETECH HLDG,CTH,2017-07-02,2016-09-30,130,2017-02-21,2018-01-24,https://www.caretech-uk.com/~/media/Files/C/Ca...,CTH.L


Company                      object
Symbol                       object
AR Date              datetime64[ns]
FY end               datetime64[ns]
YE to Report lag              int64
Period Start Date    datetime64[ns]
Period End Date      datetime64[ns]
AR link                      object
Symbol Yahoo                 object
dtype: object

In [37]:
# Get ticker symbols as list

tickers_list = sorted(tickers['Symbol Yahoo'].tolist())
    
for i, item in enumerate(tickers_list):
    print(i, item)

0 ABC.L
1 AMS.L
2 APH.L
3 ASC.L
4 BOO.L
5 BRK.L
6 BUR.L
7 CAKE.L
8 CAM.L
9 CAML.L
10 CLIN.L
11 CTH.L
12 CVSG.L
13 DTG.L
14 EMIS.L
15 FDEV.L
16 FDP.L
17 FEVR.L
18 FPM.L
19 GAMA.L
20 GBG.L
21 GHH.L
22 HCM.L
23 HUR.L
24 IOM.L
25 IQE.L
26 JHD.L
27 JSG.L
28 KWS.L
29 LTG.L
30 MIDW.L
31 MPE.L
32 MUL.L
33 NFC.L
34 NICL.L
35 NUM.L
36 POLR.L
37 PRSM.L
38 PURP.L
39 REDD.L
40 RNWH.L
41 RWS.L
42 SAA.L
43 SCPA.L
44 SIR.L
45 SMS.L
46 SOU.L
47 STAF.L
48 TEF.L
49 TFW.L
50 WINE.L
51 WJG.L
52 YNGA.L
53 YNGN.L


In [46]:
# download ticker data for at least 2 years
# Break into chunks to allow request to run
# Chunk A-E

price_data_chunk1 = pdr.get_data_yahoo(tickers_list[0:15], start="2016-01-01", end="2018-08-10")

price_data_chunk1.shape
price_data_chunk1.tail()

[*********************100%***********************]  15 of 15 downloaded


(660, 90)

Open                                                            \
             ABC.L  AMS.L      APH.L   ASC.L       BOO.L       BRK.L   BUR.L   
Date                                                                           
2018-08-06  1440.0  357.0  94.000000  6070.0  201.199997  1810.26001  1792.0   
2018-08-07  1445.0  365.0  95.000000  6080.0  200.399994  1895.00000  1810.0   
2018-08-08  1429.0  365.0  95.599998  5990.0  199.600006  1755.00000  1832.0   
2018-08-09  1452.0  352.5  95.199997  6108.0  200.199997  1775.00000  1814.0   
2018-08-10  1464.0  360.0  93.599998  6248.0  204.399994  1777.00000  1812.0   

                                    ...   Volume                        \
           CAKE.L    CAM.L CAML.L   ...    BRK.L   BUR.L  CAKE.L CAM.L   
Date                                ...                                  
2018-08-06  418.0  11650.0  230.5   ...    13171  371388  141674   221   
2018-08-07  422.5  11500.0  235.0   ...     1541  329908   69440   685   
2018-08-08  432.5  11500.0  230.0   ...     3474  349825   76055   381   
2018-08-09  430.0  11250.0  229.0   ...     2806  145378  122213   244   
2018-08-10  429.5  11500.0  226.0   ...      419  149115  206408   361   

                                                            
            CAML.L  CLIN.L  CTH.L   CVSG.L   DTG.L  EMIS.L  
Date                                                        
2018-08-06  247294  302857  27884  1284550  242934  122713  
2018-08-07  222875  114815  31555   292866  463422   67840  
2018-08-08  838348   73745  32776   261607  336204   38705  
2018-08-09  191432   66569   7126   386868  688829   98159  
2018-08-10  486098  116126  10922   258567  210785   56792  

[5 rows x 90 columns]

In [39]:
# Save api call to pickle

price_data_chunk1.to_pickle('price_data_A_E.pkl')

In [47]:
# download ticker data for at least 2 years
# Break into chunks to allow request to run
# Chunk F-L

price_data_chunk2 = pdr.get_data_yahoo(tickers_list[15:30], start="2016-01-01", end="2018-08-10")

price_data_chunk2.shape
price_data_chunk2.tail()

[*********************100%***********************]  15 of 15 downloaded


(660, 90)

Open                                                            \
            FDEV.L   FDP.L  FEVR.L       FPM.L GAMA.L  GBG.L   GHH.L   HCM.L   
Date                                                                           
2018-08-06  1145.0  4040.0  3640.0  141.800003  784.0  566.0  1505.0  4800.0   
2018-08-07  1190.0  4070.0  3550.0  137.000000  784.0  565.0  1500.0  4770.0   
2018-08-08  1165.0  4180.0  3586.0  142.800003  784.0  550.0  1545.0  4780.0   
2018-08-09  1145.0  4160.0  3589.0  140.199997  790.0  560.0  1545.0  4750.0   
2018-08-10  1070.0  4100.0  3489.0  136.199997  794.0  551.0  1550.0  4810.0   

                               ...     Volume                                 \
                HUR.L  IOM.L   ...      GBG.L  GHH.L  HCM.L    HUR.L   IOM.L   
Date                           ...                                             
2018-08-06  51.000000  393.5   ...      84300  12375   8602  2630765   55649   
2018-08-07  50.849998  395.0   ...     147602   6821   3519  4802760   51050   
2018-08-08  50.000000  394.0   ...      78840  13514   7314  4584935   95025   
2018-08-09  50.549999  398.0   ...      80201  11548  12881  4292848  106985   
2018-08-10  50.700001  404.5   ...     132497  25225  13311  2947546  123415   

                                                         
              IQE.L   JHD.L      JSG.L   KWS.L    LTG.L  
Date                                                     
2018-08-06  3630075  307820   375534.0  227168  1474450  
2018-08-07  2608891  268709   585651.0  243346  1557294  
2018-08-08  7323576  145133   161790.0  262329   630413  
2018-08-09  4886270   78410  2112304.0  216311  1277107  
2018-08-10  6428932   43319   155615.0  222630  1092126  

[5 rows x 90 columns]

In [41]:
# Save api call to pickle

price_data_chunk2.to_pickle('price_data_F_L.pkl')

In [42]:
# download ticker data for at least 2 years
# Break into chunks to allow request to run
# Chunk M-R

price_data_chunk3 = pdr.get_data_yahoo(tickers_list[30:42], start="2016-01-01", end="2018-08-10")

price_data_chunk3.shape
price_data_chunk3.tail()

[*********************100%***********************]  12 of 12 downloaded


(660, 72)

Open                                                          \
               MIDW.L  MPE.L       MUL.L  NFC.L  NICL.L  NUM.L POLR.L  PRSM.L   
Date                                                                            
2018-08-06  660.00000  754.0  596.500000  548.0  1450.5  416.5  628.0  1874.0   
2018-08-07  640.00000  734.0  598.000000  546.0  1460.0  418.5  602.0  1904.0   
2018-08-08  595.00000  740.0  585.020020  538.0  1435.0  419.5  620.0  1948.0   
2018-08-09  595.00000  752.0  607.619995  542.0  1475.0  423.0  608.0  1916.0   
2018-08-10  624.97998  749.0  572.000000  542.0  1425.0  420.0  628.0  1906.0   

                                      ...     Volume                     \
                PURP.L      REDD.L    ...      MUL.L     NFC.L   NICL.L   
Date                                  ...                                 
2018-08-06  310.000000  160.399994    ...       11.0  304469.0  10766.0   
2018-08-07  292.799988  160.000000    ...     1814.0  367716.0  10991.0   
2018-08-08  295.000000  160.000000    ...     1500.0   56513.0  18942.0   
2018-08-09  285.200012  162.000000    ...       53.0  117250.0  16476.0   
2018-08-10  285.399994  165.399994    ...     5768.0   65529.0  39173.0   

                                                                         
             NUM.L  POLR.L    PRSM.L  PURP.L  REDD.L   RNWH.L     RWS.L  
Date                                                                     
2018-08-06  152073   19139  232622.0  253698  302605  41396.0  645966.0  
2018-08-07   79524  175319  113268.0  200113  354875  43700.0  574719.0  
2018-08-08  162485   13345  109955.0  970580  435614  18767.0  408597.0  
2018-08-09   70257   18925   51403.0  164532  178267  33995.0  325081.0  
2018-08-10  152051   14344   95851.0  169719  255785  52941.0  271775.0  

[5 rows x 72 columns]

In [43]:
# Save api call to pickle

price_data_chunk3.to_pickle('price_data_M_R.pkl')

In [44]:
# download ticker data for at least 2 years
# Break into chunks to allow request to run
# Chunk S-Y

price_data_chunk4 = pdr.get_data_yahoo(tickers_list[42:], start="2016-01-01", end="2018-08-10")

price_data_chunk4.shape
price_data_chunk4.tail()

[*********************100%***********************]  12 of 12 downloaded


(660, 72)

Open                                                              \
            SAA.L      SCPA.L   SIR.L  SMS.L      SOU.L  STAF.L  TEF.L  TFW.L   
Date                                                                            
2018-08-06  365.0  436.200012  3.6613  646.0  41.700001  1128.0  408.0  302.0   
2018-08-07  370.0  438.000000  3.6613  647.0  41.200001  1166.0  406.0  301.0   
2018-08-08  362.0  429.000000  3.6613  647.0  41.000000  1160.0  410.5  295.0   
2018-08-09  356.0  420.000000  3.6613  638.0  41.400002  1142.0  413.0  302.0   
2018-08-10  359.0  431.000000  3.6613  640.0  41.000000  1142.0  408.5  304.0   

                               ...   Volume                            \
           WINE.L       WJG.L  ...    SIR.L   SMS.L   SOU.L    STAF.L   
Date                           ...                                      
2018-08-06  461.5  198.199997  ...      0.0  301881  787631   54731.0   
2018-08-07  435.5  200.500000  ...      0.0  151866  711149   23788.0   
2018-08-08  445.0  203.000000  ...      0.0  259439  661600   55201.0   
2018-08-09  428.5  201.500000  ...      0.0   95641  556256   29721.0   
2018-08-10  416.0  201.000000  ...      0.0  168906  613417  602034.0   

                                                                  
               TEF.L    TFW.L    WINE.L      WJG.L YNGA.L YNGN.L  
Date                                                              
2018-08-06   71016.0  30935.0  160354.0   149336.0  10786   8842  
2018-08-07  155055.0  39629.0   87647.0   119752.0   4506    242  
2018-08-08  101626.0  30551.0   71339.0  1075014.0   9871   4692  
2018-08-09  174052.0   9982.0  322563.0   127001.0   6245   3048  
2018-08-10  130145.0  10576.0  331579.0  2321333.0   5429   1040  

[5 rows x 72 columns]

In [45]:
# Save api call to pickle

price_data_chunk4.to_pickle('price_data_S_Y.pkl')

In [90]:
# Keep Adjusted Close and Transpose 
def trim_n_transpose(price_data_chunk, label):
    data_adj_close = price_data_chunk.iloc[:, price_data_chunk.columns.get_level_values(0)=='Adj Close']

    data_adj_close.columns = [col[1] for col in data_adj_close.columns]



    # Output to CSV for QA
    data_adj_close.to_csv("price-api-qa-" + label + ".csv")



    data_adj_close_T = data_adj_close.unstack().to_frame().reset_index()
    data_adj_close_T.columns.values
    data_adj_close_T.rename(columns={'level_0':'ticker', 0:'Adj Close'}, inplace=True)
    #data_adj_close_T.set_index(['ticker', 'Date'], inplace=True)
    
    return data_adj_close_T
    
trimmed_n_trans_chunk1 = trim_n_transpose(price_data_chunk1, "A-E")
trimmed_n_trans_chunk1.shape
trimmed_n_trans_chunk1.head()

trimmed_n_trans_chunk2 = trim_n_transpose(price_data_chunk2, "F-L")
trimmed_n_trans_chunk2.shape
trimmed_n_trans_chunk2.head()

trimmed_n_trans_chunk3 = trim_n_transpose(price_data_chunk3, "M-R")
trimmed_n_trans_chunk3.shape
trimmed_n_trans_chunk3.head()

trimmed_n_trans_chunk4 = trim_n_transpose(price_data_chunk4, "S-Y")
trimmed_n_trans_chunk4.shape
trimmed_n_trans_chunk4.head()


(9900, 3)

,ticker,Date,Adj Close
0,ABC.L,2016-01-04,625.577515
1,ABC.L,2016-01-05,635.329346
2,ABC.L,2016-01-06,628.503052
3,ABC.L,2016-01-07,626.552734
4,ABC.L,2016-01-08,626.065125


(9900, 3)

,ticker,Date,Adj Close
0,FDEV.L,2016-01-04,244.0
1,FDEV.L,2016-01-05,244.0
2,FDEV.L,2016-01-06,244.0
3,FDEV.L,2016-01-07,244.0
4,FDEV.L,2016-01-08,244.0


(7920, 3)

,ticker,Date,Adj Close
0,MIDW.L,2016-01-04,NaN
1,MIDW.L,2016-01-05,NaN
2,MIDW.L,2016-01-06,NaN
3,MIDW.L,2016-01-07,NaN
4,MIDW.L,2016-01-08,NaN


(7920, 3)

,ticker,Date,Adj Close
0,SAA.L,2016-01-04,305.043640
1,SAA.L,2016-01-05,307.967804
2,SAA.L,2016-01-06,304.575806
3,SAA.L,2016-01-07,295.218628
4,SAA.L,2016-01-08,290.540039


In [91]:
# Concat chunks together

price_data_all = pd.concat([trimmed_n_trans_chunk1, trimmed_n_trans_chunk2, trimmed_n_trans_chunk3, trimmed_n_trans_chunk4])

price_data_all.shape
price_data_all.head()

(35640, 3)

,ticker,Date,Adj Close
0,ABC.L,2016-01-04,625.577515
1,ABC.L,2016-01-05,635.329346
2,ABC.L,2016-01-06,628.503052
3,ABC.L,2016-01-07,626.552734
4,ABC.L,2016-01-08,626.065125


In [15]:
# Get indexes data for same time period - N.B. currently not returning data for AIM50 and AIM100
# FTSE 100 Index: "^FTSE"
# FTSE 250 Index: "^FTMC"
# FTSE All Share Index: "^FTAS"
# FTSE SmallCap Index: "^FTSC"
# FTSE AIM All-Share: "^FTAI"
# FTSE AIM 100 Index: "AIM1.L"
# FTSE AIM 50 Index: "AIM5.L"

tickers_indexes=["AIM1", "AIM5", "^FTSE", "^FTMC", "^FTAS", "^FTSC", "^FTAI"]

data_indexes = pdr.get_data_yahoo(tickers_indexes, start="2016-01-01", end="2018-08-10")
#data_indexes = pdr.get_data_yahoo(["AIM1.L", "AIM5.L"], start="2016-01-01", end="2018-08-10")

data_indexes.shape
data_indexes.head()

[*********************100%***********************]  7 of 7 downloaded


(660, 30)

Open                                                       \
                 ^FTAI        ^FTAS         ^FTMC        ^FTSC        ^FTSE   
Date                                                                          
2016-01-04  739.400024  3444.260010  17429.800781  4634.700195  6242.299805   
2016-01-05  736.099976  3367.419922  17122.199219  4585.700195  6093.399902   
2016-01-06  735.200012  3389.409912  17205.000000  4586.700195  6137.200195   
2016-01-07  733.500000  3356.000000  17059.000000  4566.600098  6073.399902   
2016-01-08  726.500000  3293.300049  16792.199219  4510.700195  5954.100098   

                  High                                                       \
                 ^FTAI        ^FTAS         ^FTMC        ^FTSC        ^FTSE   
Date                                                                          
2016-01-04  739.500000  3444.320068  17431.800781  4634.700195  6242.299805   
2016-01-05  736.900024  3402.620117  17223.000000  4597.799805  6166.299805   
2016-01-06  735.200012  3389.429932  17205.599609  4586.700195  6137.200195   
2016-01-07  733.500000  3356.000000  17059.000000  4566.600098  6073.399902   
2016-01-08  728.900024  3325.149902  16948.800781  4535.799805  6013.399902   

               ...       Adj Close                                          \
               ...           ^FTAI        ^FTAS         ^FTMC        ^FTSC   
Date           ...                                                           
2016-01-04     ...      735.700012  3367.419922  17122.199219  4585.700195   
2016-01-05     ...      734.599976  3389.409912  17205.000000  4586.700195   
2016-01-06     ...      734.099976  3356.000000  17059.000000  4566.600098   
2016-01-07     ...      726.099976  3293.300049  16792.199219  4510.700195   
2016-01-08     ...      725.599976  3272.610107  16732.699219  4493.299805   

                        Volume                                     
                  ^FTSE  ^FTAI ^FTAS      ^FTMC ^FTSC       ^FTSE  
Date                                                               
2016-01-04  6093.399902    0.0     0  234057900   0.0   686232700  
2016-01-05  6137.200195    0.0     0  225795300   0.0   624070800  
2016-01-06  6073.399902    0.0     0  274581200   0.0   699957700  
2016-01-07  5954.100098    0.0     0  283943900   0.0  1054315500  
2016-01-08  5912.399902    0.0     0  232393900   0.0   809964200  

[5 rows x 30 columns]

In [16]:
# Save api call to pickle
data_indexes.to_pickle('price_data_indexes.pkl')

In [17]:
# Keep Adjusted Close and Transpose 

indexes_adj_close = data_indexes.iloc[:, data_indexes.columns.get_level_values(0)=='Adj Close']

indexes_adj_close.columns = [col[1] for col in indexes_adj_close.columns]



# Output to CSV for QA
indexes_adj_close.to_csv("price-api-qa-indexes.csv")



# indexes_adj_close_T = indexes_adj_close.unstack().to_frame().reset_index()
# indexes_adj_close_T.columns.values
# indexes_adj_close_T.rename(columns={'level_0':'ticker', 0:'Adj Close'}, inplace=True)
#indexes_adj_close_T.set_index(['ticker', 'Date'], inplace=True)

indexes_adj_close.shape
indexes_adj_close.head(10)

(660, 5)

,^FTAI,^FTAS,^FTMC,^FTSC,^FTSE
Date,,,,,
2016-01-04,735.700012,3367.419922,17122.199219,4585.700195,6093.399902
2016-01-05,734.599976,3389.409912,17205.000000,4586.700195,6137.200195
2016-01-06,734.099976,3356.000000,17059.000000,4566.600098,6073.399902
2016-01-07,726.099976,3293.300049,16792.199219,4510.700195,5954.100098
2016-01-08,725.599976,3272.610107,16732.699219,4493.299805,5912.399902
2016-01-11,720.099976,3251.719971,16658.300781,4470.399902,5871.799805
2016-01-12,720.900024,3277.840088,16687.900391,NaN,5929.200195
2016-01-13,718.700012,3292.600098,16695.800781,NaN,5961.000000
2016-01-14,708.299988,3262.610107,16413.300781,NaN,5918.200195


In [ ]:
# # Loop through tickers getting data for the period dates

# def request_yahoo(row):
    
#     all_output = pd.DataFrame()
    
#     print(row['ticker'], row['startdate'], row['enddate'])
# #    return (pdr.get_data_yahoo(row['ticker'], start='2018-08-01', end='2018-08-10'))
#     day_data = pdr.get_data_yahoo(row['ticker'], start=row['startdate'], end=row['enddate'])
#     summary_df = pd.DataFrame()
#     summary_df['ticker'] = row['ticker']
#     summary_df['start_date'] = row['startdate']
#     summary_df['start_price'] = adj_close.iloc[0]
#     summary_df['end_date'] = row['enddate']
#     summary_df['end_price'] = adj_close.iloc[-1]
    
#     return summary_df

# data_all = driver_file_df.apply(request_yahoo, axis=1)

# data_all.shape
# data_all.head()

In [ ]:
# def get_prices_loop(ticker, startdate, enddate):
    
#     print("Running for ", ticker, " | startdate= ", startdate, " | enddate = ", enddate)
#     day_data = pdr.get_data_yahoo(ticker, start=startdate, end=enddate)
#     adj_close = day_data[['Adj Close']]
#     summary_df = pd.DataFrame()
# #    summary_df['ticker'] = ticker
#     summary_df['start_price'] = adj_close.iloc[0]
#     summary_df['end_price'] = adj_close.iloc[-1]
#     summary_df.insert(0, 'ticker', ticker)
#     summary_df.insert(1, 'start_date', startdate)
#     summary_df.insert(3, 'end_date', enddate)
        
#     return summary_df
    

# all_output_df = pd.DataFrame()
    
# for index, row in tickers.iterrows():
#     print(row['Symbol Yahoo'], row['Period Start Date'], row['Period End Date'])
#     output_row = get_prices_loop(row['Symbol Yahoo'], row['Period Start Date'], row['Period End Date'])
#     all_output_df = pd.concat([all_output_df, output_row])
    
# all_output_df.shape
# all_output_df.head()

## Merge datasets together

In [ ]:
#tickers.set_index(['Symbol Yahoo', 'Period Start Date']).head(30)

In [92]:
# Merge prices onto period dates file

merge_dates_prices = tickers.drop('AR link', axis=1).merge(price_data_all, how='inner', \
                        left_on=['Symbol Yahoo', 'Period Start Date'], right_on=['ticker','Date'])
merge_dates_prices.rename(columns={'Adj Close':'Adj Close Start'}, inplace=True)

merge_dates_prices.drop(['ticker', 'Date'],axis=1, inplace=True)

merge_dates_prices = merge_dates_prices.merge(price_data_all, how='inner', \
                        left_on=['Symbol Yahoo', 'Period End Date'], right_on=['ticker','Date'])
merge_dates_prices.rename(columns={'Adj Close':'Adj Close End'}, inplace=True)

merge_dates_prices.drop(['ticker', 'Date'],axis=1, inplace=True)

merge_dates_prices.shape
merge_dates_prices.head(30)

(39, 10)

,Company,Symbol,AR Date,FY end,YE to Report lag,Period Start Date,Period End Date,Symbol Yahoo,Adj Close Start,Adj Close End
0,ADVANCED MEDICAL SOLUTIONS,AMS,2017-03-14,2016-12-31,73,2017-03-28,2018-02-28,AMS.L,247.077560,324.266937
1,ALLIANCE PHARMA,APH,2017-03-29,2016-12-31,88,2017-12-04,2018-03-15,APH.L,60.762833,67.987152
2,boohoo.com,BOO,2016-05-23,2016-02-29,84,2016-06-06,2017-09-05,BOO.L,57.250000,229.500000
3,BURFORD CAPITAL,BUR,2017-03-13,2016-12-31,72,2017-03-27,2018-02-27,BUR.L,785.980713,1042.118774
4,CARETECH HLDG,CTH,2017-07-02,2016-09-30,130,2017-02-21,2018-01-24,CTH.L,365.523193,403.131958
5,CENTRAL ASIA METALS,CAML,2017-04-04,2016-12-31,94,2017-04-18,2018-03-21,CAML.L,214.977631,318.553162
6,CLINIGEN,CLIN,2016-12-10,2016-06-30,104,2016-10-26,2017-09-28,CLIN.L,735.557678,1086.528442
7,CVS GROUP,CVSG,2016-09-29,2016-06-30,91,2016-10-13,2017-09-15,CVSG.L,849.891235,1345.660645
8,DART GROUP,DTG,2016-08-16,2016-03-31,138,2016-08-30,2017-02-08,DTG.L,442.660309,503.567719
9,EMIS GROUP,EMIS,2017-03-29,2016-12-31,88,2017-12-04,2018-03-15,EMIS.L,909.857239,816.412476


In [102]:
# Calc returns

merge_dates_prices['return_ratio'] = round(merge_dates_prices['Adj Close End'] / merge_dates_prices['Adj Close Start'], 2)
merge_dates_prices['Adj Close Start'] = round(merge_dates_prices['Adj Close Start'], 1)
merge_dates_prices['Adj Close End'] = round(merge_dates_prices['Adj Close End'], 1)

merge_dates_prices.shape
merge_dates_prices.head(10)
merge_dates_prices.tail(10)

(39, 11)

,Company,Symbol,AR Date,FY end,YE to Report lag,Period Start Date,Period End Date,Symbol Yahoo,Adj Close Start,Adj Close End,return_ratio
0,ADVANCED MEDICAL SOLUTIONS,AMS,2017-03-14,2016-12-31,73,2017-03-28,2018-02-28,AMS.L,247.1,324.3,1.31
1,ALLIANCE PHARMA,APH,2017-03-29,2016-12-31,88,2017-12-04,2018-03-15,APH.L,60.8,68.0,1.12
2,boohoo.com,BOO,2016-05-23,2016-02-29,84,2016-06-06,2017-09-05,BOO.L,57.2,229.5,4.01
3,BURFORD CAPITAL,BUR,2017-03-13,2016-12-31,72,2017-03-27,2018-02-27,BUR.L,786.0,1042.1,1.33
4,CARETECH HLDG,CTH,2017-07-02,2016-09-30,130,2017-02-21,2018-01-24,CTH.L,365.5,403.1,1.10
5,CENTRAL ASIA METALS,CAML,2017-04-04,2016-12-31,94,2017-04-18,2018-03-21,CAML.L,215.0,318.6,1.48
6,CLINIGEN,CLIN,2016-12-10,2016-06-30,104,2016-10-26,2017-09-28,CLIN.L,735.6,1086.5,1.48
7,CVS GROUP,CVSG,2016-09-29,2016-06-30,91,2016-10-13,2017-09-15,CVSG.L,849.9,1345.7,1.58
8,DART GROUP,DTG,2016-08-16,2016-03-31,138,2016-08-30,2017-02-08,DTG.L,442.7,503.6,1.14
9,EMIS GROUP,EMIS,2017-03-29,2016-12-31,88,2017-12-04,2018-03-15,EMIS.L,909.9,816.4,0.90


,Company,Symbol,AR Date,FY end,YE to Report lag,Period Start Date,Period End Date,Symbol Yahoo,Adj Close Start,Adj Close End,return_ratio
29,RENEW HOLDINGS,RNWH,2016-12-14,2016-09-30,75,2016-12-28,2017-11-30,RNWH.L,423.9,415.1,0.98
30,RWS HLDGS,RWS,2017-11-01,2016-09-30,103,2017-01-25,2017-12-28,RWS.L,339.8,451.7,1.33
31,SCAPA GRP.,SCPA,2016-06-13,2016-03-31,74,2016-06-27,2017-05-30,SCPA.L,227.7,478.0,2.10
32,SECURE INCOME REIT,SIR,2017-03-29,2016-12-31,88,2017-12-04,2018-03-15,SIR.L,3.4,3.6,1.06
33,SMART METERING,SMS,2017-04-05,2016-12-31,124,2017-05-18,2018-04-20,SMS.L,563.3,797.6,1.42
34,SOUND ENERGY,SOU,2017-10-04,2016-12-31,100,2017-04-24,2018-03-27,SOU.L,74.0,48.0,0.65
35,STAFFLINE,STAF,2017-11-04,2016-12-31,101,2017-04-25,2018-03-28,STAF.L,1207.6,900.1,0.75
36,TELFORD HOMES,TEF,2016-06-20,2016-03-31,81,2016-04-07,2017-06-06,TEF.L,300.9,372.1,1.24
37,YOUNG & CO 'A',YNGA,2016-08-06,2016-03-28,72,2016-06-22,2017-05-25,YNGA.L,1205.9,1302.6,1.08
38,YOUNG & CO N/V,YNGN,2016-08-06,2016-03-28,72,2016-06-22,2017-05-25,YNGN.L,890.7,1050.7,1.18


In [104]:
# Merge on index returns in same period...

merge_tickers_indexes = merge_dates_prices.merge(indexes_adj_close.reset_index(), how='left', \
                                    left_on = 'Period Start Date', right_on='Date')

merge_tickers_indexes.rename(columns={'^FTAI':'^FTAI Start', '^FTAS':'^FTAS Start', '^FTMC':'^FTMC Start', \
                                      '^FTSC':'^FTSC Start', '^FTSE':'^FTSE Start'}, inplace=True)

merge_tickers_indexes.drop('Date',axis=1, inplace=True)

merge_tickers_indexes = merge_tickers_indexes.merge(indexes_adj_close.reset_index(), how='left', \
                                    left_on = 'Period End Date', right_on='Date')

merge_tickers_indexes.rename(columns={'^FTAI':'^FTAI End', '^FTAS':'^FTAS End', '^FTMC':'^FTMC End', \
                                      '^FTSC':'^FTSC End', '^FTSE':'^FTSE End'}, inplace=True)


merge_tickers_indexes.head(10)
merge_tickers_indexes.tail(10)

,Company,Symbol,AR Date,FY end,YE to Report lag,Period Start Date,Period End Date,Symbol Yahoo,Adj Close Start,Adj Close End,...,^FTAS Start,^FTMC Start,^FTSC Start,^FTSE Start,Date,^FTAI End,^FTAS End,^FTMC End,^FTSC End,^FTSE End
0,ADVANCED MEDICAL SOLUTIONS,AMS,2017-03-14,2016-12-31,73,2017-03-28,2018-02-28,AMS.L,247.1,324.3,...,3997.189941,18953.300781,NaN,7343.399902,2018-02-28,1038.300049,3981.610107,19687.300781,NaN,7231.899902
1,ALLIANCE PHARMA,APH,2017-03-29,2016-12-31,88,2017-12-04,2018-03-15,APH.L,60.8,68.0,...,4038.500000,19936.000000,NaN,7339.000000,2018-03-15,1055.500000,3947.159912,19828.400391,NaN,7139.799805
2,boohoo.com,BOO,2016-05-23,2016-02-29,84,2016-06-06,2017-09-05,BOO.L,57.2,229.5,...,3448.530029,17181.800781,4599.700195,6273.399902,2017-09-05,1011.299988,4044.600098,19726.800781,NaN,7372.899902
3,BURFORD CAPITAL,BUR,2017-03-13,2016-12-31,72,2017-03-27,2018-02-27,BUR.L,786.0,1042.1,...,3972.729980,18900.000000,NaN,7293.500000,2018-02-27,1043.099976,4011.520020,19875.800781,NaN,7282.500000
4,CARETECH HLDG,CTH,2017-07-02,2016-09-30,130,2017-02-21,2018-01-24,CTH.L,365.5,403.1,...,3960.169922,18772.199219,NaN,7274.799805,2018-01-24,1067.900024,4197.669922,20538.500000,NaN,7643.399902
5,CENTRAL ASIA METALS,CAML,2017-04-04,2016-12-31,94,2017-04-18,2018-03-21,CAML.L,215.0,318.6,...,3924.929932,19297.900391,NaN,7147.500000,2018-03-21,1037.300049,3895.310059,19668.000000,NaN,7039.000000
6,CLINIGEN,CLIN,2016-12-10,2016-06-30,104,2016-10-26,2017-09-28,CLIN.L,735.6,1086.5,...,3774.379883,17669.000000,NaN,6958.100098,2017-09-28,999.599976,4020.770020,19678.500000,NaN,7322.799805
7,CVS GROUP,CVSG,2016-09-29,2016-06-30,91,2016-10-13,2017-09-15,CVSG.L,849.9,1345.7,...,3789.919922,17877.099609,NaN,6977.700195,2017-09-15,993.799988,3962.439941,19378.199219,NaN,7215.500000
8,DART GROUP,DTG,2016-08-16,2016-03-31,138,2016-08-30,2017-02-08,DTG.L,442.7,503.6,...,3718.310059,17847.199219,NaN,6820.799805,2017-02-08,897.900024,3915.290039,18606.099609,NaN,7188.799805
9,EMIS GROUP,EMIS,2017-03-29,2016-12-31,88,2017-12-04,2018-03-15,EMIS.L,909.9,816.4,...,4038.500000,19936.000000,NaN,7339.000000,2018-03-15,1055.500000,3947.159912,19828.400391,NaN,7139.799805


,Company,Symbol,AR Date,FY end,YE to Report lag,Period Start Date,Period End Date,Symbol Yahoo,Adj Close Start,Adj Close End,...,^FTAS Start,^FTMC Start,^FTSC Start,^FTSE Start,Date,^FTAI End,^FTAS End,^FTMC End,^FTSC End,^FTSE End
29,RENEW HOLDINGS,RNWH,2016-12-14,2016-09-30,75,2016-12-28,2017-11-30,RNWH.L,423.9,415.1,...,3854.540039,18012.800781,NaN,7106.100098,2017-11-30,1026.500000,4033.840088,19952.900391,NaN,7326.700195
30,RWS HLDGS,RWS,2017-11-01,2016-09-30,103,2017-01-25,2017-12-28,RWS.L,339.8,451.7,...,3887.129883,18133.400391,NaN,7164.399902,2017-12-28,1045.099976,4190.330078,20642.300781,NaN,7622.899902
31,SCAPA GRP.,SCPA,2016-06-13,2016-03-31,74,2016-06-27,2017-05-30,SCPA.L,227.7,478.0,...,3237.540039,14967.900391,4272.100098,5982.200195,2017-05-30,991.099976,4119.580078,19991.900391,NaN,7526.500000
32,SECURE INCOME REIT,SIR,2017-03-29,2016-12-31,88,2017-12-04,2018-03-15,SIR.L,3.4,3.6,...,4038.500000,19936.000000,NaN,7339.000000,2018-03-15,1055.500000,3947.159912,19828.400391,NaN,7139.799805
33,SMART METERING,SMS,2017-04-05,2016-12-31,124,2017-05-18,2018-04-20,SMS.L,563.3,797.6,...,4067.860107,19691.599609,NaN,7436.399902,2018-04-20,NaN,4062.300049,20220.800781,NaN,7368.200195
34,SOUND ENERGY,SOU,2017-10-04,2016-12-31,100,2017-04-24,2018-03-27,SOU.L,74.0,48.0,...,3987.560059,19602.800781,NaN,7264.700195,2018-03-27,1021.900024,3866.919922,19389.099609,NaN,7000.100098
35,STAFFLINE,STAF,2017-11-04,2016-12-31,101,2017-04-25,2018-03-28,STAF.L,1207.6,900.1,...,3992.199951,19582.300781,NaN,7275.600098,2018-03-28,1014.000000,3885.159912,19356.599609,NaN,7044.700195
36,TELFORD HOMES,TEF,2016-06-20,2016-03-31,81,2016-04-07,2017-06-06,TEF.L,300.9,372.1,...,3370.979980,16714.199219,NaN,6136.899902,2017-06-06,975.400024,4105.930176,19654.800781,NaN,7525.000000
37,YOUNG & CO 'A',YNGA,2016-08-06,2016-03-28,72,2016-06-22,2017-05-25,YNGA.L,1205.9,1302.6,...,3437.090088,17043.500000,4545.899902,6261.200195,2017-05-25,989.099976,4114.680176,19967.599609,NaN,7517.700195
38,YOUNG & CO N/V,YNGN,2016-08-06,2016-03-28,72,2016-06-22,2017-05-25,YNGN.L,890.7,1050.7,...,3437.090088,17043.500000,4545.899902,6261.200195,2017-05-25,989.099976,4114.680176,19967.599609,NaN,7517.700195


In [105]:
# Calc index returns

merge_tickers_indexes['^FTAI_return_ratio'] = round(merge_tickers_indexes['^FTAI End'] / merge_tickers_indexes['^FTAI Start'], 2)
merge_tickers_indexes['^FTAS_return_ratio'] = round(merge_tickers_indexes['^FTAS End'] / merge_tickers_indexes['^FTAS Start'], 2)
merge_tickers_indexes['^FTMC_return_ratio'] = round(merge_tickers_indexes['^FTMC End'] / merge_tickers_indexes['^FTMC Start'], 2)
merge_tickers_indexes['^FTSC_return_ratio'] = round(merge_tickers_indexes['^FTSC End'] / merge_tickers_indexes['^FTSC Start'], 2)
merge_tickers_indexes['^FTSE_return_ratio'] = round(merge_tickers_indexes['^FTSE End'] / merge_tickers_indexes['^FTSE Start'], 2)

merge_tickers_indexes['max_index_return_ratio'] = merge_tickers_indexes[['^FTAI_return_ratio', \
                    '^FTAS_return_ratio', '^FTMC_return_ratio', '^FTSC_return_ratio', '^FTSE_return_ratio']]\
                    .max(axis=1)

merge_tickers_indexes.shape
merge_tickers_indexes[['Company', 'Symbol Yahoo', 'Period Start Date', 'Period End Date', 'return_ratio', \
                       '^FTAI_return_ratio', '^FTAS_return_ratio', '^FTMC_return_ratio', \
                       '^FTSC_return_ratio', '^FTSE_return_ratio', 'max_index_return_ratio']].head()

(39, 28)

,Company,Symbol Yahoo,Period Start Date,Period End Date,return_ratio,^FTAI_return_ratio,^FTAS_return_ratio,^FTMC_return_ratio,^FTSC_return_ratio,^FTSE_return_ratio,max_index_return_ratio
0,ADVANCED MEDICAL SOLUTIONS,AMS.L,2017-03-28,2018-02-28,1.31,1.13,1.00,1.04,NaN,0.98,1.13
1,ALLIANCE PHARMA,APH.L,2017-12-04,2018-03-15,1.12,1.03,0.98,0.99,NaN,0.97,1.03
2,boohoo.com,BOO.L,2016-06-06,2017-09-05,4.01,1.36,1.17,1.15,NaN,1.18,1.36
3,BURFORD CAPITAL,BUR.L,2017-03-27,2018-02-27,1.33,1.14,1.01,1.05,NaN,1.00,1.14
4,CARETECH HLDG,CTH.L,2017-02-21,2018-01-24,1.10,1.17,1.06,1.09,NaN,1.05,1.17


In [106]:
# Calc outperformance percentage points compared to best index performance

merge_tickers_indexes['relative_performance_pp'] = merge_tickers_indexes['return_ratio'] - merge_tickers_indexes['max_index_return_ratio']

merge_tickers_indexes[['Company', 'Symbol Yahoo', 'Period Start Date', 'Period End Date', 'return_ratio', \
                       'max_index_return_ratio', 'relative_performance_pp']].head(30)

,Company,Symbol Yahoo,Period Start Date,Period End Date,return_ratio,max_index_return_ratio,relative_performance_pp
0,ADVANCED MEDICAL SOLUTIONS,AMS.L,2017-03-28,2018-02-28,1.31,1.13,0.18
1,ALLIANCE PHARMA,APH.L,2017-12-04,2018-03-15,1.12,1.03,0.09
2,boohoo.com,BOO.L,2016-06-06,2017-09-05,4.01,1.36,2.65
3,BURFORD CAPITAL,BUR.L,2017-03-27,2018-02-27,1.33,1.14,0.19
4,CARETECH HLDG,CTH.L,2017-02-21,2018-01-24,1.10,1.17,-0.07
5,CENTRAL ASIA METALS,CAML.L,2017-04-18,2018-03-21,1.48,1.10,0.38
6,CLINIGEN,CLIN.L,2016-10-26,2017-09-28,1.48,1.21,0.27
7,CVS GROUP,CVSG.L,2016-10-13,2017-09-15,1.58,1.20,0.38
8,DART GROUP,DTG.L,2016-08-30,2017-02-08,1.14,1.13,0.01
9,EMIS GROUP,EMIS.L,2017-12-04,2018-03-15,0.90,1.03,-0.13


In [107]:
# Sort results by descending outperformance

merge_tickers_indexes[['Company', 'Symbol Yahoo', 'Period Start Date', 'Period End Date', 'return_ratio', \
                       'max_index_return_ratio', 'relative_performance_pp']].\
                    sort_values('relative_performance_pp', ascending=False)

,Company,Symbol Yahoo,Period Start Date,Period End Date,return_ratio,max_index_return_ratio,relative_performance_pp
2,boohoo.com,BOO.L,2016-06-06,2017-09-05,4.01,1.36,2.65
27,PURPLEBRICKS,PURP.L,2016-02-09,2017-05-08,3.40,1.44,1.96
17,KEYWORDS STUDIO,KWS.L,2017-07-26,2018-06-28,1.88,1.05,0.83
10,FEVERTREE,FEVR.L,2017-04-19,2018-03-22,1.89,1.09,0.80
21,MIDWICH GRP,MIDW.L,2017-04-20,2018-03-23,1.74,1.08,0.66
31,SCAPA GRP.,SCPA.L,2016-06-27,2017-05-30,2.10,1.46,0.64
18,LEARNING TECHNOLOGIES GROUP,LTG.L,2017-04-18,2018-03-21,1.72,1.10,0.62
15,IQE,IQE.L,2017-04-04,2018-07-03,1.54,1.09,0.45
13,HUTCHINSON CHINA MEDITECH LTD,HCM.L,2017-03-27,2018-02-27,1.54,1.14,0.40
26,POLAR CAPITAL,POLR.L,2016-12-07,2017-06-14,1.59,1.20,0.39


### QA missing tickers

## Scrap code

In [89]:
trimmed_n_trans_chunk3.loc[(trimmed_n_trans_chunk3['ticker'] == 'NICL.L') & (trimmed_n_trans_chunk3['Date'] == '2017-03-16')]

,ticker,Date,Adj Close
305,NICL.L,2017-03-16,1740.821533
8225,NICL.L,2017-03-16,9243.000000


In [69]:
merge_dates_prices.loc[merge_dates_prices['Symbol Yahoo'] == 'NICL.L']

,Company,Symbol,AR Date,FY end,YE to Report lag,Period Start Date,Period End Date,Symbol Yahoo,Adj Close Start,Adj Close End,return_ratio
21,NICHOLS,NICL,2017-02-03,2016-12-31,61,2017-03-16,2018-02-16,NICL.L,1740.8,1495.3,0.86
22,NICHOLS,NICL,2017-02-03,2016-12-31,61,2017-03-16,2018-02-16,NICL.L,1740.8,13856.0,7.96
23,NICHOLS,NICL,2017-02-03,2016-12-31,61,2017-03-16,2018-02-16,NICL.L,9243.0,1495.3,0.16
24,NICHOLS,NICL,2017-02-03,2016-12-31,61,2017-03-16,2018-02-16,NICL.L,9243.0,13856.0,1.50


In [75]:
temp_merge = tickers.drop('AR link', axis=1).merge(price_data_all, how='inner', \
                        left_on=['Symbol Yahoo', 'Period Start Date'], right_on=['ticker','Date'])
temp_merge.rename(columns={'Adj Close':'Adj Close Start'}, inplace=True)

temp_merge.loc[temp_merge['Symbol Yahoo'] == 'NICL.L']

,Company,Symbol,AR Date,FY end,YE to Report lag,Period Start Date,Period End Date,Symbol Yahoo,ticker,Date,Adj Close Start
26,NICHOLS,NICL,2017-02-03,2016-12-31,61,2017-03-16,2018-02-16,NICL.L,NICL.L,2017-03-16,1740.821533
27,NICHOLS,NICL,2017-02-03,2016-12-31,61,2017-03-16,2018-02-16,NICL.L,NICL.L,2017-03-16,9243.000000


In [77]:
tickers.loc[tickers['Symbol Yahoo'] == 'NICL.L']

,Company,Symbol,AR Date,FY end,YE to Report lag,Period Start Date,Period End Date,AR link,Symbol Yahoo
36,NICHOLS,NICL,2017-02-03,2016-12-31,61,2017-03-16,2018-02-16,https://www.nicholsplc.co.uk/Content/ReportPDF...,NICL.L


In [81]:
price_data_all.loc[(price_data_all['ticker'] == 'NICL.L') & (price_data_all['Date'] == '2017-03-16')]

,ticker,Date,Adj Close
305,NICL.L,2017-03-16,1740.821533
8225,NICL.L,2017-03-16,9243.000000


In [87]:
price_data_chunk4['Adj Close'].loc['2017-03-16']

SAA.L      342.311096
SCPA.L     373.293213
SIR.L        3.019439
SMS.L      581.269043
SOU.L       86.500000
STAF.L    1175.836182
TEF.L      326.914276
TFW.L      321.740662
WINE.L     329.453705
WJG.L      149.219742
YNGA.L    1329.505005
YNGN.L     977.026733
Name: 2017-03-16 00:00:00, dtype: float64

In [100]:
merge_dates_prices.loc[merge_dates_prices['Symbol Yahoo'] == 'SIR.L']

,Company,Symbol,AR Date,FY end,YE to Report lag,Period Start Date,Period End Date,Symbol Yahoo,Adj Close Start,Adj Close End,return_ratio
32,SECURE INCOME REIT,SIR,2017-03-29,2016-12-31,88,2017-12-04,2018-03-15,SIR.L,3.4,3.6,1.06


In [101]:
merge_tickers_indexes.loc[merge_tickers_indexes['Symbol Yahoo'] == 'SIR.L']


,Company,Symbol,AR Date,FY end,YE to Report lag,Period Start Date,Period End Date,Symbol Yahoo,Adj Close Start,Adj Close End,...,^FTMC End,^FTSC End,^FTSE End,^FTAI_return_ratio,^FTAS_return_ratio,^FTMC_return_ratio,^FTSC_return_ratio,^FTSE_return_ratio,max_index_return_ratio,relative_performance_pp
33,SECURE INCOME REIT,SIR,2017-03-29,2016-12-31,88,2017-12-04,2018-03-15,SIR.L,0.0,168954.0,...,19828.400391,NaN,7139.799805,1.03,0.98,0.99,NaN,0.97,1.03,inf


In [ ]:
merge_tickers_indexes.sort_values('relative_performance_pp', ascending=False)